<h2> Nav event-flow example </h2>
<h3> The following example shows how easy you can use the event-flow widget </h3>
<h3> Follow these steps: </h3>
    <ul>
        <li>Import the eventFlow method from the ntnu_process_mining library</li>
        <li> Run the eventFlow command with the path to your csv-file as an argument</li>
    </ul>

In [2]:
from ntnu_process_mining.eventFlow import eventFlow
eventFlow('sepsis.csv')

EventFlow(value=[{'ENTITY_ID': 'A', 'TS': Timestamp('2014-10-22 11:15:41+0000', tz='UTC'), 'EVENT_NAME': 'ER R…

<h2>Use the control pane to adjust the visualization to your needs </h2>
<h3>The control pane consists of five interactive settings that you can adjust</h3>
<ul>
    <li>Align sequences by: Alligns all sequnces with basis in one of the events. This makes it easier to see events leading in to an event as well as leading out from an event.</li>
    <li>Event type summary: Shows a summary of how many events it is, how many instances of each event etc. You can exclude spescific events from this setting.</li>
    <li>Hide nodes with less  than x events: Hides sequences that has less than what you input events</li>
    <li>X-axis: dictates what the x-axis should show. Either event sequence or the time that has elapsed</li>
    <li>The last setting changes the order in which the sequences is displayd down the y-axis.</li>
</ul>
